# Psycopg2

En este clase vamos a usar la librería `psycopg2`, la que nos permite conectar una base de datos PSQL con Python. Primero tenemos que conectarnos a la base de datos. Luego vamos a crear tablas e insertar valores.

In [2]:
import psycopg2

conn = psycopg2.connect("dbname=progsql user=tamara")
cur = conn.cursor()

cur.execute("DROP TABLE IF EXISTS R;")
cur.execute("CREATE TABLE R (a int, b int, PRIMARY KEY(a, b));")

cur.execute("DROP TABLE IF EXISTS S;")
cur.execute("CREATE TABLE S (b int, c int, PRIMARY KEY(b, c));")

cur.execute("INSERT INTO R VALUES(1, 1);")
cur.execute("INSERT INTO R VALUES(2, 1);")
cur.execute("INSERT INTO R VALUES(3, 1);")
cur.execute("INSERT INTO R VALUES(1, 2);")
cur.execute("INSERT INTO R VALUES(2, 2);")
cur.execute("INSERT INTO R VALUES(1, 3);")
cur.execute("INSERT INTO R VALUES(1, 4);")

cur.execute("INSERT INTO S VALUES(1, 1);")
cur.execute("INSERT INTO S VALUES(1, 2);")
cur.execute("INSERT INTO S VALUES(3, 1);")
cur.execute("INSERT INTO S VALUES(4, 1);")
cur.execute("INSERT INTO S VALUES(4, 2);")
cur.execute("INSERT INTO S VALUES(4, 3);")
cur.execute("INSERT INTO S VALUES(5, 1);")

conn.commit()

Como ejemplo veamos las siguientes consultas: 

In [3]:
cur.execute("SELECT * FROM R WHERE R.b = 1;")

t = cur.fetchone()
while t:
    print(t)
    t = cur.fetchone()
    
cur.close()

(1, 1)
(2, 1)
(3, 1)


Ahora hagamos la consulta SELECT R.b FROM R WHERE R.a =1; solo usando la consulta SELECT * FROM R;

In [4]:
cur = conn.cursor()
cur.execute("SELECT * FROM R;")

t = cur.fetchone()
while t:
    if t[0]==1:
        print(t[1])
    t = cur.fetchone()
    
cur.close()

1
2
3
4


Finalmente usando solo SELECT * FROM R; hacemos la consulta "SELECT DISTINCT R.b FROM R WHERE R.a=1"

In [5]:
cur = conn.cursor()
cur.execute("SELECT * FROM R;")


visited = []
t = cur.fetchone()
while t:
    if t[0]==1 and t[1] not in visited:
        print(t[1])
        visited.append(t[1])
    t = cur.fetchone()
    
cur.close()

1
2
3
4


Vamos a ver un ejemplo más complejo. Vamos a hacer la consulta $R \bowtie S$ suponiendo que sólo podemos hacer las consultas:

- `SELECT * FROM R`;
- `SELECT * FROM S`;

In [6]:
cur_r = conn.cursor()
cur_s = conn.cursor()

cur_r.execute("SELECT * FROM R")
cur_s.execute("SELECT * FROM S")

t_r = cur_r.fetchone()
while t_r:
    
    t_s = cur_s.fetchone()
    while t_s:
        if t_r[1] == t_s[0]:
            print("{} - {}".format(t_r, t_s))
        t_s = cur_s.fetchone()
    cur_s.execute("SELECT * FROM S")
    
    t_r = cur_r.fetchone()
    
cur_r.close()
cur_s.close()

(1, 1) - (1, 1)
(1, 1) - (1, 2)
(2, 1) - (1, 1)
(2, 1) - (1, 2)
(3, 1) - (1, 1)
(3, 1) - (1, 2)
(1, 3) - (3, 1)
(1, 4) - (4, 1)
(1, 4) - (4, 2)
(1, 4) - (4, 3)


Finalmente, siempre es muy importante cerrar la conexión.

In [7]:
conn.close()